In [12]:
def dsp(xi, xf, f_draft, f_sketch, f_proof):
    yi = f_draft(xi)
    zf = f_sketch(yi, xi, xf)
    yf = f_proof(xf, zf)
    return yf

In [13]:
import time

In [14]:
class Checker(object):
    """A modified version of the Draft, Sketch, Prove proof-checking client.
    (https://github.com/albertqjiang/draft_sketch_prove/blob/main/autoformalization/checker.py)

    This checker supports Isabelle2022 via the new version of PISA
    (https://albertqjiang.github.io/Portal-to-ISAbelle/).

    It supports checking a miniF2F-style proof via `check`.

    Finally, it replaces `sledgehammer` with a call to `normalhammer`.
    """
    def __init__(self, working_dir, isa_path, theory_file, port=9000):
        sys.path.append(os.environ['PISA_PATH'])
        try:
            from pisa_client import initialise_env
            self.initialise_env = initialise_env
        except:
            print("Set $PISA_PATH to /yourpath/to/Portal-to-ISAbelle/src/main/python")

        self.working_dir = working_dir
        self.isa_path = isa_path
        self.theory_file = theory_file
        self.port = port

    def _initialize(self):
        env = self.initialise_env(
            self.port,
            isa_path=self.isa_path,
            theory_file_path=self.theory_file,
            working_directory=self.working_dir
        )
        print("Environment initialized:", env)
        return env

    def _exit(self, env):
        try:
            env.post('exit')
        except:
            print("env.post('exit') timed out")
            pass
        os.system("ps aux | grep Isabelle | awk '{print $2}' | xargs kill -9 > /dev/null 2>&1")
        os.system("ps aux | grep poly | awk '{print $2}' | xargs kill -9 > /dev/null 2>&1")

    def _parse_output(self, obs):
        """Parse the sledgehammer output, otherwise return an empty string"""
        if '<hammer>' in obs:
            output = obs.split('<hammer>')[0]
        else:
            output = ''    
        return output

    def _run_step(self, step, i, tls_name, env):
        obs, reward, done, metadata = env.step_to_top_level_state(
            action=step,
            tls_name=tls_name,
            new_name='default_%d' % i
        )
        error = None
        if 'error:' in obs or 'Step error' in obs or 'Unknown error' in obs:
            error = obs
        return obs, reward, done, metadata, error

    def _run_sledgehammer(self, step, i, tls_name, env):
        # First try heuristics
        for heuristic in ['by auto', 'by simp', 'by blast', 'by fastforce', 'by force', 'by eval', 'by presburger', 'by sos', 'by arith', 'by linarith', 'by (auto simp: field_simps)']:
            step_ = step.replace('normalhammer', heuristic)
            obs, reward, done, metadata, error = self._run_step(step_, i, tls_name, env)
            if error is None:
                obs = '%s <hammer> %s' % (heuristic, obs)
                return obs, reward, done, metadata, error
        # Try sledgehammer
        out = self._run_step(step, i, tls_name, env)
        return out

    def check(self, statement_and_proof):
        # Initialize environment
        env = self._initialize()
        env.initialise()

        # Wrap and parse theorem
        theory = Checker.wrap_theorem(statement_and_proof)
    
        steps = Checker.get_parsed(env, theory)

        result = self._check(env, steps)
        return result

    def _check(self, env, steps):
        done = False
        reason = ''
        success = False
        step_results = []
        tls_name = 'default'
        for i, step in enumerate(steps):
            try:
                time0 = time.time()
                if 'normalhammer' in step:
                    obs, reward, done, metadata, error = self._run_sledgehammer(step, i, tls_name, env)
                else:
                    obs, reward, done, metadata, error = self._run_step(step, i, tls_name, env)
                step_time = time.time() - time0
                step_results.append(dict(index=i, step=step, output=self._parse_output(obs), step_time=step_time))
                if error is not None:
                    reason = error
                    success = False
                    done = False
                    break
            except:
                # Timeout - end the proof attempt
                success = False
                done = False
                reason = 'timeout (%d)' % len(step_results)
                step_results.append(dict(index=i, step=step, output=''))
                break

            # Change when successful
            tls_name = 'default_%d' % i

        if done and reward == 1.0:
            success = True

        result = {
            'success': success,
            'reason': reason,
            'num_steps': len(steps),
            'last_step': len(step_results),
            'step_results': step_results,
            'theorem_and_proof': self.reconstruct(step_results) if success else ''
        }
        # Exit environment
        self._exit(env)
        return result
    
    @staticmethod
    def reconstruct(step_results):
        steps = []
        for step_result in step_results[1:]:
            if step_result['output'] != '':
                steps.append(step_result['output'].strip())
            else:
                steps.append(step_result['step'].strip())
        theorem_and_proof = '\n'.join(steps)
        return theorem_and_proof

    @staticmethod
    def wrap_theorem(theorem):
        return 'theory Interactive imports HOL.HOL Complex_Main "HOL-Library.Code_Target_Numeral" "HOL-Library.Sum_of_Squares" "Symmetric_Polynomials.Vieta" "HOL-Computational_Algebra.Computational_Algebra" "HOL-Number_Theory.Number_Theory" \n begin\n%s' % theorem

    @staticmethod
    def get_parsed(env, theory, tls_name='default'):
        # HACK: the parsing doesn't work well with `normalhammer`, so we replace
        # all hammer calls with sorry, then replace sorry to normalhammer after parsing.
        theory = theory.replace('sledgehammer', 'sorry')
        theory = theory.replace('normalhammer', 'sorry')

        print("Parsing theory:", theory)

        steps = env.post(f"<parse text> ${theory}")

        print("Parsed steps:", steps)
        steps = steps.split('<SEP>')
        steps = [s for s in steps if s.strip() != '']
        # remove weird '$' step and whitespace steps
        steps = [s for s in steps if s != '$' and s.strip() != '']
        steps = [s.replace('sorry', 'normalhammer') for s in steps]
        return steps

In [15]:
import sys
import os
sys.path.append('../')
os.environ['PISA_PATH'] = '/home/siai/Portal-to-ISAbelle/src/main/python'

import dsp_utils

checker = Checker(
    working_dir='/home/siai/Isabelle2022/src/HOL/Examples',
    isa_path='/home/siai/Isabelle2022',
    theory_file='/home/siai/Isabelle2022/src/HOL/Examples/Interactive.thy',
    port=9000
)


In [16]:
import os
import sys

# Check PISA_PATH
pisa_path = os.environ.get('PISA_PATH', None)
if not pisa_path:
    print("PISA_PATH is not set.")
else:
    print(f"PISA_PATH is set to: {pisa_path}")
    if not os.path.exists(pisa_path):
        print(f"Error: The path {pisa_path} does not exist.")
    else:
        print(f"Files in PISA_PATH: {os.listdir(pisa_path)}")

# Check Isabelle paths
isa_path = '/home/siai/Isabelle2022'
working_dir = '/home/siai/Isabelle2022/src/HOL/Examples'
theory_file = '/home/siai/Isabelle2022/src/HOL/Examples/Interactive.thy'

print("\nChecking Isabelle paths:")
for path in [isa_path, working_dir, theory_file]:
    if not os.path.exists(path):
        print(f"Error: The path {path} does not exist.")
    else:
        print(f"{path} exists.")


PISA_PATH is set to: /home/siai/Portal-to-ISAbelle/src/main/python
Files in PISA_PATH: ['conjecturing_parsing', 'utils', '.idea', '__pycache__', 'legacy', 'test_client.py', 'server_pb2_grpc.py', 'server_pb2.py', 'data_extraction', 'pisa_client.py']

Checking Isabelle paths:
/home/siai/Isabelle2022 exists.
/home/siai/Isabelle2022/src/HOL/Examples exists.
/home/siai/Isabelle2022/src/HOL/Examples/Interactive.thy exists.


In [17]:
from pisa_client import initialise_env

# Initialize the environment with required parameters
env = initialise_env(
    port=9000,  # Replace with your port if not 9000
    isa_path='/home/siai/Isabelle2022',  # Path to Isabelle installation
    theory_file_path='/home/siai/Isabelle2022/src/HOL/Examples/Interactive.thy',  # Path to your Isabelle theory file
    working_directory='/home/siai/Isabelle2022/src/HOL/Examples'  # Path to Isabelle working directory
)

# Initialize the PISA-Isabelle environment
try:
    env.initialise()
    print("PISA-Isabelle communication successful.")
except Exception as e:
    print(f"Error during PISA-Isabelle initialization: {e}")

# Cleanly exit the environment
try:
    env.post('exit')
    print("Environment exited successfully.")
except Exception as e:
    print(f"Error during environment exit: {e}")


----------Path to Isabelle source----------
/home/siai/Isabelle2022
----------Path to Isabelle working directory----------
/home/siai/Isabelle2022/src/HOL/Examples
----------Path to Isabelle theory file----------
/home/siai/Isabelle2022/src/HOL/Examples/Interactive.thy
PISA-Isabelle communication successful.
Environment exited successfully.


In [18]:
theorem_and_sledgehammer_proof = """theorem simple_theorem: "(0::nat) + 1 = 1"
proof
  by simp
qed
"""

result = checker.check(theorem_and_sledgehammer_proof)

print("\n==== Success: %s" % result['success'])
print("--- Complete proof:\n%s" % result['theorem_and_proof'])

----------Path to Isabelle source----------
/home/siai/Isabelle2022
----------Path to Isabelle working directory----------
/home/siai/Isabelle2022/src/HOL/Examples
----------Path to Isabelle theory file----------
/home/siai/Isabelle2022/src/HOL/Examples/Interactive.thy
Environment initialized: <pisa_client.PisaEnv object at 0x7f1970dbe350>
Parsing theory: theory Interactive imports HOL.HOL Complex_Main "HOL-Library.Code_Target_Numeral" "HOL-Library.Sum_of_Squares" "Symmetric_Polynomials.Vieta" "HOL-Computational_Algebra.Computational_Algebra" "HOL-Number_Theory.Number_Theory" 
 begin
theorem simple_theorem: "(0::nat) + 1 = 1"
proof
  by simp
qed

Parsed steps: $<SEP>theory Interactive imports HOL.HOL Complex_Main "HOL-Library.Code_Target_Numeral" "HOL-Library.Sum_of_Squares" "Symmetric_Polynomials.Vieta" "HOL-Computational_Algebra.Computational_Algebra" "HOL-Number_Theory.Number_Theory" 
 begin<SEP>theorem simple_theorem: "(0::nat) + 1 = 1"<SEP>proof<SEP>by simp<SEP>qed

==== Success: 